# 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
import pickle

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 2. Read Dataset

In [3]:
df = pd.read_csv('../data/input/input_dataset.csv')
df.head()

,Referee,Day Of Week,Round,Days,Shots,ShotsOT,Corners,Fouls,YCards,RCards,GoalsScored,GoalsScoredHA,GoalsConceded,GoalsConcededHA,GoalsDiff,GoalsDiffHA,HTGoalsScored,HTGoalsConceded,HTGoalsScoredHA,HTGoalsConcededHA,HTGoalsDiff,HTGoalsDiffHA,Points,PointsHA,WinPercent,WinPercentHA,WinStreak,UnbPercent,UnbPercentHA,UnbStreak,Def,Mid,Att,Ovr,WinnerOdd,DrawOdd,Venue,Result
0,C Foy,Sat,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,7,6,6.33,-7.10,4.5,Emirates Stadium,D
1,R East,Sat,1.0,-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,-1,-2,-4,-2.33,-1.20,3.3,Swansea.com Stadium,D
2,L Mason,Sat,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,-1.0,-3,-2,-2,-2.33,-1.50,3.3,Boleyn Ground,D
3,L Probert,Sat,1.0,-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,7,8,8,7.67,-9.71,5.5,Etihad Stadium,W
4,M Halsey,Sat,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,-1.0,1,1,-2,0.00,-2.25,3.3,Bet365 Stadium,D


In [4]:
df.shape

(3800, 38)

In [5]:
df.head()

,Referee,Day Of Week,Round,Days,Shots,ShotsOT,Corners,Fouls,YCards,RCards,GoalsScored,GoalsScoredHA,GoalsConceded,GoalsConcededHA,GoalsDiff,GoalsDiffHA,HTGoalsScored,HTGoalsConceded,HTGoalsScoredHA,HTGoalsConcededHA,HTGoalsDiff,HTGoalsDiffHA,Points,PointsHA,WinPercent,WinPercentHA,WinStreak,UnbPercent,UnbPercentHA,UnbStreak,Def,Mid,Att,Ovr,WinnerOdd,DrawOdd,Venue,Result
0,C Foy,Sat,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,7,6,6.33,-7.10,4.5,Emirates Stadium,D
1,R East,Sat,1.0,-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,-1,-2,-4,-2.33,-1.20,3.3,Swansea.com Stadium,D
2,L Mason,Sat,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,-1.0,-3,-2,-2,-2.33,-1.50,3.3,Boleyn Ground,D
3,L Probert,Sat,1.0,-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,7,8,8,7.67,-9.71,5.5,Etihad Stadium,W
4,M Halsey,Sat,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,-1.0,1,1,-2,0.00,-2.25,3.3,Bet365 Stadium,D


In [6]:
df = df.dropna(axis=0).reset_index(drop=True)
df.head()

,Referee,Day Of Week,Round,Days,Shots,ShotsOT,Corners,Fouls,YCards,RCards,GoalsScored,GoalsScoredHA,GoalsConceded,GoalsConcededHA,GoalsDiff,GoalsDiffHA,HTGoalsScored,HTGoalsConceded,HTGoalsScoredHA,HTGoalsConcededHA,HTGoalsDiff,HTGoalsDiffHA,Points,PointsHA,WinPercent,WinPercentHA,WinStreak,UnbPercent,UnbPercentHA,UnbStreak,Def,Mid,Att,Ovr,WinnerOdd,DrawOdd,Venue,Result
0,P Dowd,Wed,0.0,-1,4.8,2.0,2.3,-2.2,-0.3,0.0,0.4,0.4,-0.4,-0.4,8.0,8.0,0.3,0.1,-0.1,-0.2,2.0,1.0,0.4,0.5,10.0,20.0,1.0,20.0,10.0,1.0,5,5,2,4.00,-7.10,4.5,Anfield,W
1,A Taylor,Sat,0.0,0,-1.0,-2.2,-0.6,0.6,0.3,0.1,-0.1,0.3,0.6,0.3,-7.0,0.0,-0.4,0.0,0.0,0.1,-4.0,-1.0,-0.5,0.0,-20.0,0.0,-1.0,-10.0,0.0,0.0,0,-5,-5,-3.33,-1.65,3.4,DW Stadium,L
2,L Mason,Sat,0.0,-1,-5.0,-4.3,-1.0,-0.1,0.3,0.0,-0.3,0.7,0.5,0.2,-8.0,5.0,-0.6,0.2,-0.2,0.2,-8.0,-4.0,-0.5,0.3,-20.0,0.0,0.0,-10.0,30.0,0.0,-4,-5,-4,-4.33,-0.50,3.4,Madejski Stadium,W
3,R East,Sat,0.0,0,2.0,0.1,0.9,-0.1,0.0,0.1,0.1,0.2,0.1,-0.6,0.0,8.0,-0.4,0.0,-0.2,-0.3,-4.0,1.0,-0.2,0.4,-10.0,10.0,1.0,0.0,20.0,1.0,-3,0,-3,-2.00,-1.95,3.5,The Hawthorns,W
4,M Jones,Sat,0.0,0,0.1,-1.0,-0.9,-0.7,0.1,0.0,-0.6,-0.6,-0.1,-0.4,-5.0,-2.0,-0.5,0.0,-0.6,0.0,-5.0,-6.0,-0.1,-0.2,-10.0,-20.0,1.0,10.0,20.0,0.0,0,-1,2,0.34,-1.25,3.4,Loftus Road,W


In [7]:
df.shape

(3324, 38)

# 3. Splitting to Train and Test Set

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA

## 3.1 Data Split 1

- Target Variable: Draw & Non-Draw
- Sampling: Stratified

In [9]:
draw_df = df[df['Result'] == 'D']
non_draw_df = df[(df['Result'] == 'W') | (df['Result'] == 'L')]
non_draw_df['Result'] = non_draw_df['Result'].map({"W": "ND", "L": "ND"})

print(draw_df.Result.value_counts())
print(non_draw_df.Result.value_counts())

D    775
Name: Result, dtype: int64
ND    2549
Name: Result, dtype: int64


C:\Users\choibs\AppData\Local\Temp\ipykernel_21088\2887461011.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_draw_df['Result'] = non_draw_df['Result'].map({"W": "ND", "L": "ND"})


In [10]:
# Setting up 5 fold_df
splits = [pd.DataFrame(columns=list(df.columns)) for i in range(5)]

In [11]:
# Setting up draw_df
draw_df = draw_df.sample(frac = 1, random_state=0)

split_sizes = [155,155,155,155,155]

for i in range(5):
    # copy section of draw_df to fold_df
    splits[i] = pd.concat([splits[i], draw_df.iloc[:split_sizes[i], :]])
    draw_df = draw_df.drop(draw_df.index[:split_sizes[i]])

In [12]:
# Setting up non_draw_df
non_draw_df = non_draw_df.sample(frac=1, random_state=0)

split_sizes = [509,510,510,510,510]

for i in range(5):
    # copy section of non_draw_df to fold_df
    splits[i] = pd.concat([splits[i], non_draw_df.iloc[:split_sizes[i], :]])
    non_draw_df = non_draw_df.drop(non_draw_df.index[:split_sizes[i]])

In [13]:
# save to csv files
for i in range(5):
    # prepare test set
    test_df = splits[i]
    test_df = test_df.sample(frac=1, random_state=0).reset_index(drop=True) # randomly shuffle test_df
    test_df.to_csv(f'data-split-1/fold-{i+1}/test.csv', index=False) # save to test.csv
    
    # prepare train set
    train_df = pd.DataFrame(columns=list(df.columns))
    for j in range(5):
        if (j != i):
            train_df = pd.concat([train_df, splits[j]])
    
    train_df = train_df.sample(frac=1, random_state=0).reset_index(drop=True) # randomly shuffle train_df
    train_df.to_csv(f'data-split-1/fold-{i+1}/train.csv', index=False) # save to train.csv

## 3.2 Data Split 2

- Target Variable: Draw & Non-Draw
- Sampling: Balanced

In [14]:
draw_df = df[df['Result'] == 'D']
non_draw_df = df[(df['Result'] == 'W') | (df['Result'] == 'L')]
non_draw_df['Result'] = non_draw_df['Result'].map({"W": "ND", "L": "ND"})

print(draw_df.Result.value_counts())
print(non_draw_df.Result.value_counts())

D    775
Name: Result, dtype: int64
ND    2549
Name: Result, dtype: int64


C:\Users\choibs\AppData\Local\Temp\ipykernel_21088\2887461011.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_draw_df['Result'] = non_draw_df['Result'].map({"W": "ND", "L": "ND"})


In [15]:
d_range = [(0, 620), (40, 660), (79, 699), (118, 738), (155, 775)]
nd_range = [(0, 620), (482, 1102), (964, 1584), (1447, 2067), (1929, 2549)]

for i in range(5):
    test_df = pd.DataFrame(columns=list(df.columns))
    train_df = pd.DataFrame(columns=list(df.columns))
    
    # prepare d_df and nd_df
    d_df = draw_df.sample(frac = 1, random_state=0)
    nd_df = non_draw_df.sample(frac=1, random_state=0)
    
    # copy section of d_df to train set
    d_start = d_range[i][0]
    d_end = d_range[i][1]
    train_df = pd.concat([train_df, d_df.iloc[d_start:d_end, :]])
    d_df = d_df.drop(d_df.index[d_start:d_end])
    
    # copy section of nd_df to train set
    nd_start = nd_range[i][0]
    nd_end = nd_range[i][1]
    train_df = pd.concat([train_df, nd_df.iloc[nd_start:nd_end, :]])
    nd_df = nd_df.drop(nd_df.index[nd_start:nd_end])
    
    # prepare test set
    test_df = pd.concat([test_df, d_df])
    test_df = pd.concat([test_df, nd_df])
    
    # save to csv files
    train_df = train_df.sample(frac=1, random_state=0).reset_index(drop=True) # randomly shuffle train_df
    train_df.to_csv(f'data-split-2/fold-{i+1}/train.csv', index=False)
    
    test_df = test_df.sample(frac=1, random_state=0).reset_index(drop=True) # randomly shuffle test_df
    test_df.to_csv(f'data-split-2/fold-{i+1}/test.csv', index=False)

## 3.3 Data Split 3

- Target Variable: Win, Draw, Lose
- Sampling: Stratified

In [16]:
win_df = df[df['Result'] == 'W']
draw_df = df[df['Result'] == 'D']
lose_df = df[df['Result'] == 'L']

print(win_df.Result.value_counts())
print(draw_df.Result.value_counts())
print(lose_df.Result.value_counts())

W    1490
Name: Result, dtype: int64
D    775
Name: Result, dtype: int64
L    1059
Name: Result, dtype: int64


In [17]:
# Setting up 5 fold_df
splits = [pd.DataFrame(columns=list(df.columns)) for i in range(5)]

In [18]:
# Setting up win_df
win_df = win_df.sample(frac = 1, random_state=0)

split_sizes = [298,298,298,298,298]

for i in range(5):
    # copy section of win_df to fold_df
    splits[i] = pd.concat([splits[i], win_df.iloc[:split_sizes[i], :]])
    win_df = win_df.drop(win_df.index[:split_sizes[i]])

In [19]:
# Setting up draw_df
draw_df = draw_df.sample(frac = 1, random_state=0)

split_sizes = [155,155,155,155,155]

for i in range(5):
    # copy section of draw_df to fold_df
    splits[i] = pd.concat([splits[i], draw_df.iloc[:split_sizes[i], :]])
    draw_df = draw_df.drop(draw_df.index[:split_sizes[i]])

In [20]:
# Setting up lose_df
lose_df = lose_df.sample(frac = 1, random_state=0)

split_sizes = [211,212,212,212,212]

for i in range(5):
    # copy section of lose_df to fold_df
    splits[i] = pd.concat([splits[i], lose_df.iloc[:split_sizes[i], :]])
    lose_df = lose_df.drop(lose_df.index[:split_sizes[i]])

In [21]:
# save to csv files
for i in range(5):
    # prepare test set
    test_df = splits[i]
    test_df = test_df.sample(frac=1, random_state=0).reset_index(drop=True) # randomly shuffle test_df
    test_df.to_csv(f'data-split-3/fold-{i+1}/test.csv', index=False)
    
    # prepare train set
    train_df = pd.DataFrame(columns=list(df.columns))
    for j in range(5):
        if (j != i):
            train_df = pd.concat([train_df, splits[j]])
    
    train_df = train_df.sample(frac=1, random_state=0).reset_index(drop=True) # randomly shuffle train_df
    train_df.to_csv(f'data-split-3/fold-{i+1}/train.csv', index=False)

## 3.4 Data Split 4

- Target Variable: Win, Draw, Lose
- Sampling: Balanced

In [22]:
win_df = df[df['Result'] == 'W']
draw_df = df[df['Result'] == 'D']
lose_df = df[df['Result'] == 'L']

print(win_df.Result.value_counts())
print(draw_df.Result.value_counts())
print(lose_df.Result.value_counts())

W    1490
Name: Result, dtype: int64
D    775
Name: Result, dtype: int64
L    1059
Name: Result, dtype: int64


In [23]:
w_range = [(0, 620), (217, 837), (435, 1055), (655, 1275), (870, 1490)]
d_range = [(0, 620), (40, 660), (79, 699), (118, 738), (155, 775)]
l_range = [(0, 620), (111, 731), (220, 840), (330, 950), (439, 1059)]

for i in range(5):
    test_df = pd.DataFrame(columns=list(df.columns))
    train_df = pd.DataFrame(columns=list(df.columns))
    
    # prepare d_df and nd_df
    w_df = win_df.sample(frac = 1, random_state=0)
    d_df = draw_df.sample(frac=1, random_state=0)
    l_df = lose_df.sample(frac=1, random_state=0)
    
    # copy section of w_df to train set
    w_start = w_range[i][0]
    w_end = w_range[i][1]
    train_df = pd.concat([train_df, w_df.iloc[w_start:w_end, :]])
    w_df = w_df.drop(w_df.index[w_start:w_end])
    
    # copy section of d_df to train set
    d_start = d_range[i][0]
    d_end = d_range[i][1]
    train_df = pd.concat([train_df, d_df.iloc[d_start:d_end, :]])
    d_df = d_df.drop(d_df.index[d_start:d_end])
    
    # copy section of l_df to train set
    l_start = l_range[i][0]
    l_end = l_range[i][1]
    train_df = pd.concat([train_df, l_df.iloc[l_start:l_end, :]])
    l_df = l_df.drop(l_df.index[l_start:l_end])
    
    # prepare test set
    test_df = pd.concat([test_df, w_df])
    test_df = pd.concat([test_df, d_df])
    test_df = pd.concat([test_df, l_df])
    
    # save to csv files
    train_df = train_df.sample(frac=1, random_state=0).reset_index(drop=True) # randomly shuffle train_df
    train_df.to_csv(f'data-split-4/fold-{i+1}/train.csv', index=False)
    
    test_df = test_df.sample(frac=1, random_state=0).reset_index(drop=True) # randomly shuffle test_df
    test_df.to_csv(f'data-split-4/fold-{i+1}/test.csv', index=False)

## 3.5 Data Split 5

- Target Variable: Draw, Lose
- Sampling: Stratified

In [24]:
draw_df = df[df['Result'] == 'D']
lose_df = df[df['Result'] == 'L']

print(draw_df.Result.value_counts())
print(lose_df.Result.value_counts())

D    775
Name: Result, dtype: int64
L    1059
Name: Result, dtype: int64


In [25]:
# Setting up 5 fold_df
splits = [pd.DataFrame(columns=list(df.columns)) for i in range(5)]

In [26]:
# Setting up draw_df
draw_df = draw_df.sample(frac = 1, random_state=0)

split_sizes = [155,155,155,155,155]

for i in range(5):
    # copy section of draw_df to fold_df
    splits[i] = pd.concat([splits[i], draw_df.iloc[:split_sizes[i], :]])
    draw_df = draw_df.drop(draw_df.index[:split_sizes[i]])

In [27]:
# Setting up lose_df
lose_df = lose_df.sample(frac = 1, random_state=0)

split_sizes = [211,212,212,212,212]

for i in range(5):
    # copy section of lose_df to fold_df
    splits[i] = pd.concat([splits[i], lose_df.iloc[:split_sizes[i], :]])
    lose_df = lose_df.drop(lose_df.index[:split_sizes[i]])

In [28]:
# save to csv files
for i in range(5):
    # prepare test set
    test_df = splits[i]
    test_df = test_df.sample(frac=1, random_state=0).reset_index(drop=True) # randomly shuffle test_df
    test_df.to_csv(f'data-split-5/fold-{i+1}/test.csv', index=False)
    
    # prepare train set
    train_df = pd.DataFrame(columns=list(df.columns))
    for j in range(5):
        if (j != i):
            train_df = pd.concat([train_df, splits[j]])
    
    train_df = train_df.sample(frac=1, random_state=0).reset_index(drop=True) # randomly shuffle train_df
    train_df.to_csv(f'data-split-5/fold-{i+1}/train.csv', index=False)

## 3.6 Data Split 6

- Target Variable: Draw, Lose
- Sampling: Balanced

In [29]:
draw_df = df[df['Result'] == 'D']
lose_df = df[df['Result'] == 'L']

print(draw_df.Result.value_counts())
print(lose_df.Result.value_counts())

D    775
Name: Result, dtype: int64
L    1059
Name: Result, dtype: int64


In [30]:
d_range = [(0, 620), (40, 660), (79, 699), (118, 738), (155, 775)]
l_range = [(0, 620), (111, 731), (220, 840), (330, 950), (439, 1059)]

for i in range(5):
    test_df = pd.DataFrame(columns=list(df.columns))
    train_df = pd.DataFrame(columns=list(df.columns))
    
    # prepare d_df and nd_df
    d_df = draw_df.sample(frac=1, random_state=0)
    l_df = lose_df.sample(frac=1, random_state=0)
    
    # copy section of d_df to train set
    d_start = d_range[i][0]
    d_end = d_range[i][1]
    train_df = pd.concat([train_df, d_df.iloc[d_start:d_end, :]])
    d_df = d_df.drop(d_df.index[d_start:d_end])
    
    # copy section of l_df to train set
    l_start = l_range[i][0]
    l_end = l_range[i][1]
    train_df = pd.concat([train_df, l_df.iloc[l_start:l_end, :]])
    l_df = l_df.drop(l_df.index[l_start:l_end])
    
    # prepare test set
    test_df = pd.concat([test_df, d_df])
    test_df = pd.concat([test_df, l_df])
    
    # save to csv files
    train_df = train_df.sample(frac=1, random_state=0).reset_index(drop=True) # randomly shuffle train_df
    train_df.to_csv(f'data-split-6/fold-{i+1}/train.csv', index=False)
    
    test_df = test_df.sample(frac=1, random_state=0).reset_index(drop=True) # randomly shuffle test_df
    test_df.to_csv(f'data-split-6/fold-{i+1}/test.csv', index=False)